# Custom Report Demo

This demo shows how to create and query a dataset. The dataset in this case is generated by running an RCSB PDB web service to create a custom report of PDB annotations.

[PDB custom report](http://www.rcsb.org/pdb/results/reportField.do)

## Imports

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import customReportService

#### Configure Spark 

In [2]:
spark = SparkSession.builder.master("local[4]").appName("CustomReportDemo").getOrCreate()

## Retrieve PDB annotation
Binding affinities (Ki, Kd), group name of the ligand (hetId), and the SMILES string of the ligand.

In [3]:
ds = customReportService.get_dataset(["Ki","Kd","hetId","ligandSmiles"])

## Show the schema of this dataset

In [4]:
ds.printSchema()

root
 |-- structureChainId: string (nullable = true)
 |-- structureId: string (nullable = true)
 |-- chainId: string (nullable = true)
 |-- Ki: string (nullable = true)
 |-- Kd: string (nullable = true)
 |-- hetId: string (nullable = true)
 |-- ligandSmiles: string (nullable = true)



## Filtering

### Select structures that either have Ki or Kd values(s)

In [5]:
ds = ds.filter("Ki IS NOT NULL OR Kd IS NOT NULL")

ds.sample(fraction = 0.01, seed = 1).show(10)

+----------------+-----------+-------+--------------------+--------------------+-----+--------------------+
|structureChainId|structureId|chainId|                  Ki|                  Kd|hetId|        ligandSmiles|
+----------------+-----------+-------+--------------------+--------------------+-----+--------------------+
|          1A0T.R|       1A0T|      R|                null|50000000 (PDBbind...|  SUC|C([C@@H]1[C@H]([C...|
|          1A4L.D|       1A4L|      D|    0.001-0.01 (BDB)|                null|  DCF|c1nc2c(n1[C@H]3C[...|
|          1A7T.A|       1A7T|      A|  23000000 (PDBbind)|                null|  MES|C1COCC[NH+]1CCS(=...|
|          1A9R.A|       1A9R|      A|                null|           680 (BDB)|  HPA|c1[nH]c2c(n1)N=CN...|
|          1BGO.A|       1BGO|      A|       3.5 (PDBbind)|                null|  I10|CC(C)C[C@H](c1ccc...|
|          1BR5.A|       1BR5|      A|2000000 (BMOAD_15...|                null|  NEO|c1c(nc2c(n1)N=C(N...|
|          1CNY.A|       1CN

## Terminate Spark

In [6]:
spark.stop()